In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
%pwd

'/home/bath/data/gadgets'

In [3]:
dat = pd.read_csv('/home/bath/data/ald_mask.csv')

In [4]:
dat = dat.replace (np.nan,'',regex=True) 
dat['Fuse'] = dat['AT'].map(str)+";"+dat['MT'].map(str)
dat['Fuse']

0                                  自身免疫性肝炎; 气血; 贾建伟;专家意见
1                         自身免疫性肝炎; 医案; 干燥综合征; 滋水清肝饮;病例报告
2                           自身免疫性肝病; 肝硬化;原发性胆汁性胆管炎; 专家意见
3                    ;治疗组; 原发性胆汁性肝硬化; 病例报告; 临床分析; 复方甘草酸苷
4      ;liver cirrhosis; 肝硬化; 理论探讨; 胆汁性; 毒损胆汁络; 微观病机;...
                             ...                        
509                                  自身免疫性肝炎; 高月求; 从肾论治;
510                             原发性硬化性胆管炎; 中西医结合; 茵陈五苓散;
511                中医药; 数据挖掘; 医案; 网络药理学; 原发性胆汁性肝硬化; 陈四清;
512                        自身免疫性肝病; 中西医结合; 证候; 病下分证; 诊治;
513                                    肝脾同治; 名中医经验; 常占杰;
Name: Fuse, Length: 514, dtype: object

In [5]:
dat['fuse2'] = dat['Fuse'].replace('[a-z\sA-Z]+;','', regex=True)

In [6]:
s1 = dat['Fuse'].str.split(";")

In [7]:
s2 = []
for item in s1:
    #去掉留白，空白，和手动添加或无意义的关键词
    new_l = [key.strip() for key in item]
    new_l = [key for key in new_l if key != '']
    out = ['综述','专家意见','动物研究','细胞研究','队列研究','综述','随机对照研究','理论文章','排除','病例报告','病例对照研究','中医']
    for w in out:
        new_l = [key for key in new_l if w not in key]
    #只保留中文关键词
    new = []
    for kw in new_l:
        flag = re.compile(u"[\u4e00-\u9fa5]+")
        res = re.search(flag,kw)
        if res:
            new.append(kw)
    #压平列表
    final = ';'.join(new)
    s2.append(final)

In [8]:
flag = re.compile(u"[\u4e00-\u9fa5]+")
s = "inflammation"
res = re.search(flag,s)
print(res)
if res:
    print("Found")
    print(type(res))
    print(res.group())
else:
    print("Not Found")

None
Not Found


In [9]:
old = ['自身免疫性肝炎', '刀豆蛋白A', '枫杨', '肝损伤', '炎症', 'concanavalin A', 'inflammation', 'liver injury', 'Pterocarya hupehensis Skan']
new = []
for kw in old:
    print(kw)
    flag = re.compile(u"[\u4e00-\u9fa5]+")
    res = re.search(flag,kw)
    if res:
        new.append(kw)
print(new)

自身免疫性肝炎
刀豆蛋白A
枫杨
肝损伤
炎症
concanavalin A
inflammation
liver injury
Pterocarya hupehensis Skan
['自身免疫性肝炎', '刀豆蛋白A', '枫杨', '肝损伤', '炎症']


In [10]:
dat['Final'] = pd.Series(s2)
dat.drop(['Fuse','fuse2'], axis=1, inplace=True)

In [11]:
dat.drop(['MT','AT'],axis =1, inplace=True)

In [20]:
with open('clean.txt','w+') as f:
    for i in range(514):
        f.write("RT {}\nA1 {}\nT1 {}\nJF {}\nK1 {}\nYR {}\nLA {}\nDS {}\n".format(dat['RT'][i],dat['A1'][i],dat['T1'][i],dat['JF'][i],dat['Final'][i],dat['YR'][i],
             dat['LA'][i],dat['DS'][i]))

In [16]:
dat.to_excel('clean.xlsx', encoding='utf-8', index=False)

In [13]:
dat.to_csv('ald_for_r.csv', encoding='utf-8', index=False)